# Load data
We included 5 different datasets in our work:
3 synthetic ones
Military press (MP)
Counter Movement Jump (CMJ)

In [1]:
from load_data import load_data

only accepted arguments for dataset_name are "synth","CMJ" and "MP" 

The returned value is a dictionary containing both x any for both train and test set

In [2]:
datasets = load_data(data_name="synth",concat=False)

In [3]:
datasets["PseudoPeriodic_Positional_False"].keys()

dict_keys(['X_train', 'X_test', 'y_train', 'y_test', 'meta_train', 'meta_test'])

In [4]:
X_train = datasets["PseudoPeriodic_Positional_False"]['X_train']
X_test = datasets["PseudoPeriodic_Positional_False"]['X_test']
y_train = datasets["PseudoPeriodic_Positional_False"]['y_train']
y_test = datasets["PseudoPeriodic_Positional_False"]['y_test']

# TRAIN MODELS
## train rocket concatenated

In [5]:
#load concat data
import numpy as np
concat_data = load_data(data_name="synth",concat=True)

# loading the data using concat=True provide a 2D array 
# but sktime is expacting a 3D one also in univariate case

X_train_concat = np.expand_dims(
    concat_data["PseudoPeriodic_Positional_False"]['X_train'],1)
X_test_concat =  np.expand_dims(
    concat_data["PseudoPeriodic_Positional_False"]['X_test'],1)
y_train_concat = concat_data["PseudoPeriodic_Positional_False"]['y_train']
y_test_concat = concat_data["PseudoPeriodic_Positional_False"]['y_test']

In [6]:
from sktime.transformations.panel.rocket import Rocket
from sklearn.linear_model import LogisticRegressionCV
from  sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [7]:
cls = make_pipeline(Rocket(normalise=True,n_jobs=-1) ,StandardScaler(),
 LogisticRegressionCV(cv = 5, random_state=0, n_jobs = -1,max_iter=1000))


In [8]:
cls.fit(X_train_concat, y_train_concat)

Pipeline(steps=[('rocket', Rocket(n_jobs=-1)),
                ('standardscaler', StandardScaler()),
                ('logisticregressioncv',
                 LogisticRegressionCV(cv=5, max_iter=1000, n_jobs=-1,
                                      random_state=0))])

In [9]:
acc = cls.score(X_test_concat, y_test_concat)
print("Rocket accuracy is ",acc)

Rocket accuracy is  1.0


## train dResNet


In [10]:
from dCAM.src.models.CNN_models import dResNetBaseline,ModelCNN
from utilities import transform_data4ResNet

In [11]:
# change your device to cpu if you don't have CUDA
device="cuda"

In [12]:
train_dataloader,test_dataloader, n_channels, n_classes, device,_ = transform_data4ResNet(
        datasets["PseudoPeriodic_Positional_False"],
    "PseudoPeriodic_Positional_False",concat=False)

In [13]:
modelarch = dResNetBaseline(n_channels,mid_channels=64,
        num_pred_classes=n_classes).to(device)

In [14]:
# "n_epochs_stop" parameter is used for early stopping
# i.e. (number of not improving accuracy epochs before stop)
dResNet = ModelCNN(model=modelarch 
            ,n_epochs_stop=20,device=device)

In [15]:
# "num_epochs" is maximum epochs number
acc = dResNet.train(num_epochs=100,train_loader=train_dataloader,test_loader=test_dataloader)

Epoch [1/100], Loss Train: 0.6795,Loss Test: 0.6922, Accuracy Train: 54.00%, Accuracy Test: 74.00%
Epoch [11/100], Loss Train: 0.6093,Loss Test: 0.6145, Accuracy Train: 84.00%, Accuracy Test: 87.00%
Epoch [21/100], Loss Train: 0.4808,Loss Test: 0.4875, Accuracy Train: 95.00%, Accuracy Test: 87.00%
Epoch [31/100], Loss Train: 0.3792,Loss Test: 0.3897, Accuracy Train: 96.00%, Accuracy Test: 93.00%
Epoch [41/100], Loss Train: 0.3171,Loss Test: 0.3269, Accuracy Train: 98.00%, Accuracy Test: 94.00%
Epoch [51/100], Loss Train: 0.2545,Loss Test: 0.2857, Accuracy Train: 99.00%, Accuracy Test: 95.00%
Epoch [61/100], Loss Train: 0.2494,Loss Test: 0.2622, Accuracy Train: 100.00%, Accuracy Test: 97.00%
Epoch [71/100], Loss Train: 0.2528,Loss Test: 0.2360, Accuracy Train: 100.00%, Accuracy Test: 98.00%
Epoch [81/100], Loss Train: 0.1870,Loss Test: 0.2206, Accuracy Train: 100.00%, Accuracy Test: 98.00%
Epoch [91/100], Loss Train: 0.1908,Loss Test: 0.2012, Accuracy Train: 100.00%, Accuracy Test: 98.0

# EXPLAIN
## dCAM

In [16]:
X_specimen = X_test[0]
y_specimen = (y_test[0])

In [17]:
from dCAM.src.explanation.DCAM import DCAM

In [18]:
# need to identify last convolutional layer and 
#  fully connected layer 
last_conv_layer = dResNet.model._modules['layers'][2]
fc_layer_name = dResNet.model._modules['final']

In [19]:
dcam = DCAM(dResNet.model,device,last_conv_layer=last_conv_layer,fc_layer_name=fc_layer_name)

In [20]:
dcam,permutation_success = dcam.run(
    instance=X_specimen, nb_permutation=200, 
    label_instance=y_specimen,generate_all=False)

  0%|                                                   | 0/200 [00:00<?, ?it/s]/home/davide/workspace/PhD/Trang/first_experiment/dCAM/src/explanation/DCAM.py:133: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred_probabilities = F.softmax(prediction).data.squeeze()
100%|█████████████████████████████████████████| 200/200 [00:12<00:00, 16.40it/s]


In [21]:
# dCAM result is a 2D heat map
dcam.shape

(20, 100)

## timeXplain

In [22]:
import sys
sys.path.insert(0, 'timeXplain')
import timexplain as tx
from sklearn.preprocessing import FunctionTransformer

/home/davide/miniconda/envs/last_torch/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
concat_data = load_data(data_name="synth",concat=True)

X_test_concat = concat_data["PseudoPeriodic_Positional_False"]['X_test']
y_test_concat = concat_data["PseudoPeriodic_Positional_False"]['y_test']

In [25]:
X_specimen = X_test_concat[0]
y_specimen = y_test_concat[0]

size_x = X_specimen.shape[-1]

# We want 10 slices for each of  the 20 channels = 
# 200 tot slices
n_slices = 200

In [27]:
# timeXplain expects 1D inputs. We need to have as first 
# pipeline step np.expand_dims
cls = make_pipeline(  FunctionTransformer( lambda x: np.expand_dims(x,1), validate=True),
    cls.steps[0][1],cls.steps[1][1],cls.steps[2][1])

In [28]:
rocket_om = tx.om.TimeSliceOmitter(size_x, 
    time_slicing=n_slices, x_repl=tx.om.x_sample)

shap_explainer = tx.om.KernelShapExplainer(rocket_om,
    cls.predict_proba, X_bg=X_test_concat, y_bg=y_test_concat, 
            n_samples=500, n_builds=5, bgcs=True)


In [29]:
rocket_expl = shap_explainer.explain(X_specimen)

  0%|                                                     | 0/2 [00:00<?, ?it/s]

  0%|                                                     | 0/5 [00:00<?, ?it/s]

 20%|█████████                                    | 1/5 [00:09<00:38,  9.58s/it]

 40%|██████████████████                           | 2/5 [00:19<00:29,  9.99s/it]

 60%|███████████████████████████                  | 3/5 [00:30<00:20, 10.29s/it]

 80%|████████████████████████████████████         | 4/5 [00:41<00:10, 10.54s/it]

100%|█████████████████████████████████████████████| 5/5 [00:52<00:00, 10.65s/it]

                                                                                
 50%|██████████████████████▌                      | 1/2 [00:52<00:52, 52.29s/it]

  0%|                                                     | 0/5 [00:00<?, ?it/s]

 20%|█████████                                    | 1/5 [00:11<00:44, 11.13s/it]

 40%|██████████████████                           | 2/5 [00:22<00:33, 11.13s/it]

 60%|████████████

In [30]:
rocket_expl.impacts.shape
# explanation  shape is now (#n classes,n_slices)
#To have a multivariate explanation we need to reshape

(2, 200)

In [31]:
multi_rocket_expl = np.reshape(
    rocket_expl.impacts,(2,20,-1))
print( multi_rocket_expl.shape)

(2, 20, 10)
